Импортируем модули

In [1]:
import time
import datetime

import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

Считываем входные данные

In [2]:
X_train_raw = pd.read_csv("features.csv")
X_test_raw = pd.read_csv("features_test.csv")

X_train = X_train_raw
X_test = X_test_raw
y = X_train["radiant_win"]

---

# Подход 1: градиентный бустинг "в лоб"

**1.** Удаялем признаки, связанные с итогами матчей (в test их нет, поэтому удаляем только из train, кэп)

In [3]:
X_train = X_train.drop([
        'tower_status_radiant',
        'duration',
        'tower_status_dire',
        'radiant_win',
        'barracks_status_dire',
        'barracks_status_radiant'
    ], axis=1)

**2.** Проверяем выборку на наличие NaN. Считаем их количество. Выделяем признаки, в которых они имеются. Пытаемся объяснить их наличие.

In [4]:
X_all = pd.concat((X_train, X_test), axis=0) # т.к. нас интересует вся выборка, объединяем train и test

В задании говорится про ф-цию "count()", но по-моему "isnull()" - куда удобнее. Получаем количество nan по каждому из признаков.

In [5]:
sum_nan_by_column = X_all.isnull().sum()
sum_nan_by_column = sum_nan_by_column[sum_nan_by_column > 0]
sum_nan_by_column = sum_nan_by_column.sort_values(ascending=False)
print(sum_nan_by_column)

first_blood_player2            51753
radiant_flying_courier_time    32364
dire_flying_courier_time       30622
first_blood_player1            23105
first_blood_team               23105
first_blood_time               23105
dire_bottle_time               18985
radiant_bottle_time            18586
radiant_first_ward_time         2166
dire_first_ward_time            2089
radiant_courier_time             819
dire_courier_time                806
dtype: int64


В абсолютных величинах это выглядит не очень репрезентативно, перейдем к относительным. <br/>
Посчитаем сколько NaN в % во всей выборке и в каждом признаке отдельно.

In [6]:
perc_nan_in_X = sum_nan_by_column.sum() / (X_all.shape[0] * X_all.shape[1])
print("%-30s: %f" % ("Percentage NaN in X", perc_nan_in_X * 100.0))

print("")
print("Percentage NaN by column")

len_rows = len(X_all)
for i in range(len(sum_nan_by_column)):
    print("%-30s: %f" % (sum_nan_by_column.keys()[i], (sum_nan_by_column[i] / len_rows)*100.0))

Percentage NaN in X           : 1.930639

Percentage NaN by column
first_blood_player2           : 45.235868
radiant_flying_courier_time   : 28.288479
dire_flying_courier_time      : 26.765845
first_blood_player1           : 20.195443
first_blood_team              : 20.195443
first_blood_time              : 20.195443
dire_bottle_time              : 16.594264
radiant_bottle_time           : 16.245509
radiant_first_ward_time       : 1.893241
dire_first_ward_time          : 1.825937
radiant_courier_time          : 0.715865
dire_courier_time             : 0.704502


Как видно относительно всей выборки число NaN не так уж велико - 1.93%. А вот для отдельных признаков весьма... <br/>
Касательно природы появления NaN: <br/>

- "first_blood_player2" (45% NaN) - по данному признаку ставится NaN, в случае если оно не успело произойти за время измерения(первые 5 минут). Поэтому можно сделать вывод, что данное событие происходит не часто или не так быстро, относительно начала игры. Пытливый data scientist задумается, а почему по "first_blood_player2" более чем в два раза (45% vs 20%) больше NaN, чем по "first_blood_player1"? Я думаю это связано с тем, что второй (и другие игроки) не всегда бывают причастны к этому событию.<br/><br/>
- "radiant_flying_courier_time" (28% NaN) - тут все просто: данный предмет не был приобретен за первые 5 минут.   

**3.** Заполняем пропуски

- **3.1** Нулями

In [7]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

- **3.2** Чем-то очень большим|маленьким (для деревьев все пропуски уйдут в отдельную вершину)

- **3.3** Чем-то средним (среднее, медиана, мода)

**4.** Целевую переменную (исход матча) **radiant_win** положили в переменную **y** ранее.

In [8]:
print(y.shape, X_train.shape)

(97230,) (97230, 103)


**5.** Обучение модели. Попробуем количество деревьев от 10 до 50 с шагом в 10. 

In [9]:
cv = KFold(len(X_train), shuffle=True, n_folds=5, random_state=32)
n_estimators_variants = np.linspace(10, 50, num=5)
n_estimators_scores = list(range(len(n_estimators_variants)))

for i, n_estimators in enumerate(n_estimators_variants):
    start_time = datetime.datetime.now()
    model = GradientBoostingClassifier(n_estimators=int(n_estimators), random_state=32)
    scores = cross_val_score(model, X_train, y=y, cv=cv, scoring='roc_auc')
    n_estimators_scores[i] = scores.mean()
    print('N trees:', n_estimators, 
          'Score:', n_estimators_scores[i], 
          'Time elapsed:', datetime.datetime.now() - start_time)


N trees: 10.0 Score: 0.664038510869 Time elapsed: 0:00:46.638668
N trees: 20.0 Score: 0.682470555294 Time elapsed: 0:01:35.881484
N trees: 30.0 Score: 0.689438923221 Time elapsed: 0:02:45.735480
N trees: 40.0 Score: 0.694169429123 Time elapsed: 0:04:28.299346
N trees: 50.0 Score: 0.697676711145 Time elapsed: 0:05:07.358580


Обучение, кросс-валидация и оценка - происходили за приемлемое время. Результаты и время расчетов приведены выше.<br/>
Для обучения 30 деревьев нам потребовалось около 2.5 минут, при этом на кросс-валидации согласно метрике качества AUC-ROC получилось ~ 0.69. Значение **n_estimators=30** можно считать оптимальным. Так как, при его увеличении качество увеличивается незначительно, относительно времени вычислений. Ускорить обучение можно следующим образом: использовать методы понижения размерности (PCA), обучаться на случайном подмножестве выборки, ограничить глубину дерева, использовать библиотеки, включающие методы параллельных вычисление (XGBoost). 

---

# Подход 2: логистическая регрессия

**1.** Обучаем модель и подбираем коэффициент регуляризации (C).

In [10]:
def train_logistic_regression(X_train, y):
    X_train_scaled = scale(X_train)
    cv = KFold(len(X_train), shuffle=True, n_folds=5, random_state=32)
    C_variants = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
    C_scores = list(range(len(C_variants)))
    for i, C in enumerate(C_variants):
        start_time = datetime.datetime.now()
        model = LogisticRegression(penalty='l2', C=C, random_state=32)
        scores = cross_val_score(model, X_train_scaled, y=y, cv=cv, scoring='roc_auc')
        C_scores[i] = scores.mean()
        print("%-5s %-15f %-5s %-15f %-5s %s" % ('C value:', C, 
                                      'Score:', C_scores[i],
                                      'Time elapsed:', datetime.datetime.now() - start_time))
    best_score = max(C_scores)
    best_C = C_variants[C_scores.index(best_score)]
    print("")
    print("Best C value:", best_C, 'Score:', best_score)
    return best_score, best_C

In [11]:
best_score, best_C = train_logistic_regression(X_train, y)

C value: 0.000010        Score: 0.695115        Time elapsed: 0:00:03.087176
C value: 0.000100        Score: 0.711304        Time elapsed: 0:00:04.492257
C value: 0.001000        Score: 0.716346        Time elapsed: 0:00:07.681440
C value: 0.010000        Score: 0.716562        Time elapsed: 0:00:10.207584
C value: 0.100000        Score: 0.716544        Time elapsed: 0:00:10.480599
C value: 1.000000        Score: 0.716540        Time elapsed: 0:00:10.381593
C value: 10.000000       Score: 0.716540        Time elapsed: 0:00:12.172697
C value: 100.000000      Score: 0.716540        Time elapsed: 0:00:11.892680
C value: 1000.000000     Score: 0.716540        Time elapsed: 0:00:11.982685

Best C value: 0.01 Score: 0.716561747302


C:\Users\Kirill\Desktop\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Лучшее качество (~0.72) достигается при C=0.01, что немного лучше, в сравнении с бустингом проведенным ранее. Расчеты выполняется быстрее, это наглядно видно выше. Улучшение качетсва может быть обусловлено следующими причинами:<br/>
- В сравнение с логистического регрессией, решающие деревья более подвержены переобучению, но этого можно избежать использую прунинг.
- Решающие деревья основываются на том, что разделющие гиперплоскости будут строго параллельны координатным осям, а логистическая регрессия не имеет такого предположения. Поэтому можно сделать вывод, что выборка разделяется "линейной" гиперплоскостью, не являющейся параллельной осям, лучше, чем множеством гиперплоскостей параллельных осям координат.<br/>

*Источник: https://www.quora.com/What-are-the-advantages-of-logistic-regression-over-decision-trees *

**2.** Убираем категориальные признаки и заново обучаемся

In [12]:
X_train = X_train.drop([
        'lobby_type',
        'r1_hero',
        'r2_hero',
        'r3_hero',
        'r4_hero',
        'r5_hero',
        'd1_hero',
        'd2_hero',
        'd3_hero',
        'd4_hero',
        'd5_hero'], axis=1)

In [13]:
best_score, best_C = train_logistic_regression(X_train, y)

C value: 0.000010        Score: 0.695066        Time elapsed: 0:00:02.480142
C value: 0.000100        Score: 0.711306        Time elapsed: 0:00:03.728213
C value: 0.001000        Score: 0.716394        Time elapsed: 0:00:06.981399
C value: 0.010000        Score: 0.716608        Time elapsed: 0:00:09.435540
C value: 0.100000        Score: 0.716589        Time elapsed: 0:00:10.140580
C value: 1.000000        Score: 0.716585        Time elapsed: 0:00:10.131579
C value: 10.000000       Score: 0.716584        Time elapsed: 0:00:10.113579
C value: 100.000000      Score: 0.716584        Time elapsed: 0:00:10.051575
C value: 1000.000000     Score: 0.716584        Time elapsed: 0:00:10.052575

Best C value: 0.01 Score: 0.71660786983


C:\Users\Kirill\Desktop\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


После удаления категориальных признаков качество немного возросло (0.7165 vs 0.7166), при этом Best C Value=0.01 осталось прежним. Объяснить данное улучшение можно следующим образом:<br/>
- *"lobby_type"* - тип комнаты, в которой собираются игроки, носит чисто формальный характер, так как в описании процесса игры я не нашел информации о влиянии типа выбранной комнаты на механику игры.
- *"r|d_1-5_hero"* - тип соответствующего игрока. Наверняка есть определенные стратегии(правила) выбора игроков и запрета выбора для другой команды (определяемые игроками или внутренними механизмами игры), которые не позволяют получить значимый перевес одной из команд при выборе игроков и существенным образом повлиять на исход матча. То есть, разумеется, наверняка, есть такие типы игроков, которые выигрывают чаще чем другие, но мы ведь рассматриваем не просто одного игрока, а их комбинацию, потому и их значимость начинает шуметь. Возможно, стоит формировать фичи из самых часто встречающихся комбинаций и работать уже с ними.<br/>

Таким образом, влияние данных признаков можно считать несущественным, что и подтверждают результаты обучения модели. 

**3.** Считаем типы героев

In [14]:
heroes = X_all[[
        'r1_hero',
        'r2_hero',
        'r3_hero',
        'r4_hero',
        'r5_hero',
        'd1_hero',
        'd2_hero',
        'd3_hero',
        'd4_hero',
        'd5_hero']]

heroes = heroes.dropna()
unic_heroes = pd.Series(heroes.values.ravel()).unique()
print(len(unic_heroes))

108


Получили 108 уникальных типов героев. Однако если посмотреть файл "heroes.csv", то там их **112** (последний индекс 113, но строчек 112). Значит какие-то герои не очень популярны =)<br/>
*Проверка: http://dota2.gamepedia.com/Heroes_by_release*

**4.** Кодирование информации о героях

In [15]:
X_train = X_train_raw.drop([
        'tower_status_radiant',
        'duration',
        'tower_status_dire',
        'radiant_win',
        'barracks_status_dire',
        'barracks_status_radiant',
        'lobby_type'], axis=1)

X_train = X_train.fillna(0)

def get_hero_features(X):
    heros_amount = 112
    X_pick = np.zeros((X.shape[0], heros_amount))
    for i, match_id in enumerate(X.index):
        for p in range(5):
            X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    X_pick_df = pd.DataFrame(X_pick, columns=['hero_' + str(n) for n in range(1, heros_amount + 1)])
    return X_pick_df

X_train = pd.concat((X_train, get_hero_features(X_train)), axis=1)

**5.** Проведем кросс-валидацию на новой выборке

In [16]:
best_score, best_C = train_logistic_regression(X_train, y)

C value: 0.000010        Score: 0.714910        Time elapsed: 0:00:04.333248
C value: 0.000100        Score: 0.742964        Time elapsed: 0:00:06.960398
C value: 0.001000        Score: 0.751864        Time elapsed: 0:00:13.331762
C value: 0.010000        Score: 0.752191        Time elapsed: 0:00:18.354050
C value: 0.100000        Score: 0.752154        Time elapsed: 0:00:19.976143
C value: 1.000000        Score: 0.752147        Time elapsed: 0:00:20.781188
C value: 10.000000       Score: 0.752147        Time elapsed: 0:00:20.815191
C value: 100.000000      Score: 0.752147        Time elapsed: 0:00:20.812191
C value: 1000.000000     Score: 0.752147        Time elapsed: 0:00:20.795190

Best C value: 0.01 Score: 0.752190530631


C:\Users\Kirill\Desktop\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Качество выросло: **0.717 vs 0.752**. Раньше алгоритм пытался разделить выборку на два класса в пространстве, значения которого были от 1 до 112, воспринимая их как непрерывную величину. Что является не совсем верным, относительно природы данного признака, так как его значения являются категориальными.

**6.** Строим предсказания вероятностей победы команды Radiant для тестовой выборки

In [17]:
X_test = X_test.drop('lobby_type', axis=1)
X_test = pd.concat((X_test, get_hero_features(X_test)), axis=1)


clf = LogisticRegression(penalty='l2', C=0.01, random_state=32)
clf.fit(scale(X_train), y)
predictions = clf.predict_proba(scale(X_test))

C:\Users\Kirill\Desktop\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\sklearn\preprocessing\data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [18]:
print("%-35s: %f" % ('Минимальное значение предсказания', predictions.min()))
print("%-35s: %f" % ('Максимальное значение предсказания', predictions.max()))

Минимальное значение предсказания  : 0.003591
Максимальное значение предсказания : 0.996409


---